In [54]:
# import modules

import win32gui
import win32api
import numpy as np
from PIL import ImageGrab, Image, ImageDraw
import cv2
import pandas as pd

In [ ]:
# set constants and define functions

block_width, block_height = 20, 20

def SegmentBlock(img, x, y):
    """
    Segment the board into blocks
    args:
        img: game board image grabbed
        x, y: position of the block to be cropped
    returns:
        img_cropped: the single block cropped
    """
    x1, y1 = x*block_height, y*block_width
    x2, y2 = x1+block_height, y1+block_width
    img_cropped = img.crop((y1, x1, y2, x2))
    return img_cropped

def exist_rgb(rgb_array, img_rgb):
    """
    Check if certain color exists in the input image
    args:
        rgb_array: the rgb representation of the color being checked
        img_rgb: the input image to check with
    returns:
        boolean: either the color exists
    """
    for i in range(len(img_rgb)):
        for element in img_rgb[i]:
            if np.array_equal(rgb_array, element):
                return True
    else:
        return False

def identify_block(blocks, blocks_y, blocks_x):
    """
    Identify each block and repressent the board in the form of 2D numpy array of ints.
    Different blocks are converted into int values as following:
        1-8: number on the block
        9: flagged
        -1: unclicked
        0: blank
        -2: mine
    args:
        blocks: 2D array of blocks
        blocks_y, blocks_x: dimension of blocks
    returns:
        mine_block: 2D numpy array of ints
    """
    category_block = []
    category_block.append(np.array([192,192,192])) # unclicked
    category_block.append(np.array([255,0,0])) # 1
    category_block.append(np.array([0,128,0])) # 2
    category_block.append(np.array([0,0,255])) # 3
    category_block.append(np.array([128,0,0])) # 4
    category_block.append(np.array([0,0,128])) # 5
    category_block.append(np.array([128,128,0])) # 6
    category_block.append(np.array([0,0,0])) # 7
    category_block.append(np.array([114,114,114])) # 8
    category_block.append(np.array([255,255,255])) # white

    mine_block = [[0 for i in range(blocks_x)] for i in range(blocks_y)]
    for x in range(blocks_x):
        for y in range(blocks_y):
            block = blocks[x][y]
            img_rgb = cv2.cvtColor(np.array(block), cv2.COLOR_BGR2RGB)
            if exist_rgb(category_block[1], img_rgb):
                mine_block[x][y] = 1
            elif exist_rgb(category_block[2], img_rgb):
                mine_block[x][y] = 2
            elif exist_rgb(category_block[3], img_rgb):
                if not exist_rgb(category_block[7], img_rgb):
                    mine_block[x][y] = 3
                elif np.array_equal(np.array([0,0,255]), img_rgb[3,4]):
                    mine_block[x][y] = -2
                else:
                    mine_block[x][y] = 9
            elif exist_rgb(category_block[4], img_rgb):
                mine_block[x][y] = 4
            elif exist_rgb(category_block[5], img_rgb):
                mine_block[x][y] = 5
            elif exist_rgb(category_block[6], img_rgb):
                mine_block[x][y] = 6
            elif exist_rgb(category_block[7], img_rgb):
                if exist_rgb(category_block[9], img_rgb):
                        mine_block[x][y] = -2
                else:
                    mine_block[x][y] = 7
            elif exist_rgb(category_block[8], img_rgb):
                mine_block[x][y] = 8
            elif exist_rgb(category_block[9], img_rgb):
                mine_block[x][y] = -1
            else:
                mine_block[x][y] = 0
    return mine_block

def ConcatBlock(img, block, x, y):
    """
    Paste the blocks back into an image of board
    args:
        img: image of board
        block: block to be pasted to img
        x, y: position of the block to be pasted
    """
    x1, y1 = x*block_height, y*block_width
    img.paste(block, (y1, x1))

In [72]:
# Get the window of minesweeper

class_name = "TMain" # class of the window
title_name = "Minesweeper Arbiter" # title of the window
hwnd = win32gui.FindWindow(class_name, title_name)

if hwnd:
    left, top, right, bottom = win32gui.GetWindowRect(hwnd)
    # print(str(left)+" "+str(top)+" "+str(right)+" "+str(bottom))
else:
    print("Can't find minesweeper")

In [73]:
# Grab the board and preprocessing

# Adjust the rect
left += 44
top += 178
right += 78
bottom += 98
rect = (left, top, right, bottom)
img = ImageGrab.grab().crop(rect)

# Compute number of blocks on each edge
blocks_x = int((right-left)/block_width)
blocks_y = int((bottom-top)/block_height)

# Segment the image
blocks = [[0 for i in range(blocks_x)] for i in range(blocks_y)]

for x in range(blocks_x):
    for y in range(blocks_y):
        blocks[x][y] = SegmentBlock(img, x, y)

In [76]:
# Represent current stage in the form of a 2D numpy array

current = identify_block(blocks, blocks_y, blocks_x)

In [69]:
# Visualize the current stage

df = pd.DataFrame(current)
# print("Current minesweeper board")
# print(df)

In [77]:
# Find possible mines and mark on the image

current = np.array(current)
marked = np.copy(current)
while True:
    temp = np.copy(marked)
    for i in range(1, len(current[0])-1):
        for j in range(1, len(current)-1):
            block = marked[j-1:j+2, i-1:i+2]
            center = marked[j][i]
            count_unclicked = np.count_nonzero(block == -1)
            count_flagged = np.count_nonzero(block == 9)
            if (count_unclicked+count_flagged) == center:
                if np.any(block == -1):
                    pos_list = np.argwhere(block == -1)
                    for pos in pos_list:
                        draw = ImageDraw.Draw(blocks[pos[0]+j-1][pos[1]+i-1])
                        draw.ellipse((3, 3, 17, 17), fill=(255, 0, 0), outline=(0, 0, 0))
                block[block==-1] = 9
            elif count_flagged == center:
                block[block==-1] = 0
            marked[j-1:j+2, i-1:i+2] = block
    if np.array_equal(temp, marked):
        break

# Visualize the prediction
df_marked = pd.DataFrame(marked)
# print("Marked with possible mines")
# print(df_marked)


In [78]:
# Concatinate the blocks into one image with markers

board = Image.new('RGB', (img.width, img.height))

for x in range(blocks_x):
    for y in range(blocks_y):
        ConcatBlock(board, blocks[x][y], x, y)

board.show()